In [1]:
import wave
import numpy as np
import math
import wave
import os
import struct
import matplotlib.pyplot as plt


def freq_to_number(f):
    return 69 + 12 * np.log2(f / 440.0)


NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


def note_name(n):
    return NOTE_NAMES[n % 12] + str(int(n / 12 - 1))

In [14]:
def individual_note_detect(audio_file):
    audio_file = wave.open(audio_file)
    # -------------------------------------------
    # here we are just storing our sound file as a numpy array
    # you can also use any other method to store the file as an np array
    file_length = audio_file.getnframes()
    counter = audio_file.getnchannels()  # number of channels mono/sterio
    print(counter)
    f_s = audio_file.getframerate()  # sampling frequency
    sound = np.zeros(file_length)  # blank array



    for i in range(file_length):
        wdata = audio_file.readframes(1)
        data = struct.unpack("<h", wdata)
        sound[i] = int(data[0])


    curr = 0
    times = []
  

    i = 0


    windowlen = 2000

    CAP = 1000

    sound_avg = 300

    while i < (len(sound) - windowlen - 200):
        if (sound[i] < 0):
            i += 1; continue
        else:
            if (sound[i+windowlen]-sound[i] >= CAP):
                times.append(i)
                i += windowlen * 15
        i += 1
 

    sound = np.divide(sound, float(2**15))  # scaling it to 0 - 1
    # -------------------------------------------

    fourier = np.fft.fft(sound)
    fourier = np.absolute(fourier)
    imax = np.argmax(fourier[0 : int(file_length / 2)])  # index of max element

    #  reference: https://github.com/Amagnum/Music-notes-detection
   
    i_begin = -1
    threshold = 0.3 * fourier[imax]
    for i in range(0, imax + 100):
        if fourier[i] >= threshold:
            if i_begin == -1:
                i_begin = i
        if i_begin != -1 and fourier[i] < threshold:
            break
    i_end = i
    imax = np.argmax(fourier[0 : i_end + 100])

    freq = (imax * f_s) / (file_length * counter)  # formula to convert index into sound frequency

    note = note_name(round(freq_to_number(freq)))

    
    return note

In [22]:
def full_note_detect(file_path):
    audio_file = wave.open(file_path)
    
    file_length = audio_file.getnframes()
    counter = audio_file.getnchannels()  # number of channels mono/sterio
    f_s = audio_file.getframerate()  # sampling frequency
    sound = np.zeros(file_length)  # blank array

    assert counter == 1 


    for i in range(file_length):
        wdata = audio_file.readframes(1)
    
        data = struct.unpack("<h", wdata)
        sound[i] = int(data[0])

    
    windowlen = 2000

    CAP = 1000

    sound_avg = 300
    
    times = []    
    i = 0

    while i < (len(sound) - windowlen - 200):
        if (sound[i] < 0):
            i += 1; continue
        else:
            if (sound[i+windowlen]-sound[i] >= CAP):
                times.append(i)
                i += windowlen * 15
        i += 1
    
    print("number of notes: ", len(times))

    
    timesinsecs = []
    for i in range(len(times) - 1):
        start_time = times[i]/f_s
        end_time = (times[i+1] - 200)/f_s
        timesinsecs.append((start_time, end_time))
    
    start_time = times[len(times) - 1]/f_s
    end_time = len(sound)
    timesinsecs.append((start_time, end_time))
    
    
    os.system("mkdir parts")
    list_of_notes = []
    
    os.system("rm -rf parts/*.wav")
    
    import subprocess
    import time
    
    for i in range(len(timesinsecs)):
        start_time = timesinsecs[i][0]
        start_time = math.floor(start_time * 10)/10
        end_time = timesinsecs[i][1]
        end_time = math.floor(end_time * 10)/10
        cmd_string = f"ffmpeg -ss {start_time} -i {file_path} -t {(end_time - start_time)} -c copy parts/{i}.wav > /dev/null"
        os.popen(cmd_string)
       
        time.sleep(1)
        list_of_notes.append(individual_note_detect(f"parts/{i}.wav"))
        
    print(list_of_notes)
    print(timesinsecs)

In [23]:
full_note_detect("hbd2.wav")

number of notes:  13


mkdir: cannot create directory ‘parts’: File exists
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-

1


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1


...

1


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1
['A2', 'G4', 'G4', 'A4', 'G4', 'C5', 'B4', 'G4', 'G4', 'A4', 'G4', 'D5', 'C5']
[(0.80475, 3.065354166666667), (3.0695208333333333, 4.108), (4.112166666666667, 5.2482291666666665), (5.252395833333333, 6.290708333333333), (6.294875, 7.380541666666667), (7.384708333333333, 8.607333333333333), (8.6115, 10.985958333333333), (10.990125, 12.091270833333333), (12.0954375, 13.215583333333333), (13.21975, 14.280291666666667), (14.284458333333333, 15.336625), (15.340791666666666, 16.358041666666665), (16.362208333333335, 885976)]


a

In [ ]:
'G4', 'G4', 'A4', 'G4', 'C5', 'B4', 'G4', 'G4', 'A4', 'G4', 'D5', 'C5']

a

a